In [1]:
import pandas as pd
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizerSumLegal = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
modelSumLegal = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")
summarizerFacebook = pipeline("summarization", model="facebook/bart-large-cnn")
tokenizerFaceBook = summarizerFacebook.tokenizer

################### LAWMARKCASES ##################################
FILE_LAWMARK = '/home/jaco/Projetos/landMarkClassification/data/cornellLandmarks/raw/scrapedData.csv'

dfLawMark = pd.read_csv(FILE_LAWMARK)

dfLawMark['text'] = dfLawMark['text'].astype(str)
dfLawMark['subject'] = dfLawMark['subject'].astype(str)

#tirar numeros? fazer uma limpeza? Tirar os 300 nomes de advogados q tem? Tirar os nomes das leis?
#tirar os numeros q marcam os paragrafos? Syllabus e Opinion ja resolvem?

def limpeza_basica(texto):

    texto = texto['text']

    texto = texto[169:-124] ### tirando uns html que nao sairam

    texto = re.sub('\. [0-9]+ ', ' ', texto)
    texto = re.sub('\* \* \*', ' ', texto)
    texto = re.sub('\.\' [0-9]+ ', ' ', texto)
    texto = re.sub('\.+ \.+ \.+', ' ', texto)
    texto = re.sub('\[.+?omitted\]', '', texto)
    texto = re.sub('"\." [0-9]+ ', ' ', texto)
    texto = re.sub('\' [0-9]+ [a-z]', ' ', texto,flags=re.IGNORECASE) #deixar todos com essa parada de ignore case?

    texto = re.sub('"\." [0-9]+ ', ' ', texto)

    texto = re.sub('\.[0-9]+ ', '.', texto) #pode mexer muito, muito especifico
    texto = re.sub('\. [0-9]+ ', ' ', texto) #pode mexer muito, muito especifico

    texto = re.sub(' +',' ',texto)

    return texto

dfLawMark['text'] = dfLawMark.apply(lambda row : limpeza_basica(row), axis=1)

#dfLawMark.to_csv('/home/jaco/Projetos/landMarkClassification/data/checkAplicacaoRegex.csv',index=False)

dfLawMark = dfLawMark[['text','subject']]

dfLawMark = dfLawMark[(dfLawMark['text'] != '')] ### checar depois pq aparece isso, o scrape pode retornar nulo? 
dfLawMark = dfLawMark[dfLawMark["text"].apply(lambda x: len(x) > 1000)] ### checar depois pq aparece isso, pq tem processos que vem do scraper que nao puxa quase nada?
dfLawMark = dfLawMark.sample(frac=1)

print(f'tamanho do dataset: {len(dfLawMark)}')

##testando classificações mais facieis:
## tirando classes sem samples suficientes:
min_sample_size = 14
df2 = dfLawMark.groupby(['subject'],as_index=False).count()
df2 = df2[df2['text'] >= min_sample_size]
lista = list(df2['subject'])

#####
#dfLawMark = dfLawMark[dfLawMark['subject'].isin(lista)]
dfLawMark = dfLawMark[dfLawMark['subject'].isin(['First Amendment','Freedom of Speech','Criminal Law','Racial Discrimination'])]
#####

DATSET_SIZE = len(dfLawMark)
ROW_COUNTER = 0

print(f'tamanho do dataset resumido: {len(dfLawMark)}')

### checar se o webscrape ta perfeito vs o site.
### pegar um sumarizador profissional law:

/home/jaco/.pyenv/versions/3.9.7/envs/landMarkClassification/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tamanho do dataset: 2271
tamanho do dataset resumido: 393


In [2]:
len(lista)

49

In [3]:
dfLawMark.head(5)
#dfLawMark = dfLawMark.head(5)

,text,subject
1166,"461 U.S 103 S.Ct 75 L.Ed.2d 708 Harry CONNICK,...",Freedom of Speech
1099,"354 U.S 77 S.Ct 1 L.Ed.2d 1498 Samuel ROTH, Pe...",Freedom of Speech
836,403 U.S 91 S.Ct 29 L.Ed.2d 745 Alton J. LEMON ...,First Amendment
789,"297 U.S 56 S.Ct 80 L.Ed GROSJEAN, Sup'r of Pub...",First Amendment
793,310 U.S 60 S.Ct 84 L.Ed CANTWELL et al.v.STATE...,First Amendment


In [4]:
def summary_legal_text(row):

    global ROW_COUNTER

    row = get_syllabus(row,char_size=75200)

    input_tokenized = tokenizerSumLegal.encode(row, return_tensors='pt',padding="max_length",pad_to_max_length=True, max_length=int((16384)*0.98),truncation=True)

    summary_ids = modelSumLegal.generate(input_tokenized,
                                    num_beams=4,
                                    no_repeat_ngram_size=3,
                                    length_penalty=2,
                                    min_length=256,
                                    max_length=512
                                    )

    summary = [tokenizerSumLegal.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

    print(f'% Indiano completo : {round(ROW_COUNTER/DATSET_SIZE*100,2)} ')

    ROW_COUNTER = ROW_COUNTER + 1

    summary = re.sub('Â', '', summary) #?

    return summary

def summary_facebook(row):

    global ROW_COUNTER

    row = get_syllabus(row,char_size=4700)
    row = tokenizerFaceBook(row,padding='max_length', max_length = int((1024)*0.98), truncation=True, return_tensors="pt")
    row = tokenizerFaceBook.decode(row.input_ids[0], skip_special_tokens=True)

    row = summarizerFacebook(row, max_length=512, min_length=256, do_sample=False)
    
    print(f'% Facebook completo : {round(ROW_COUNTER/DATSET_SIZE*100,2)} ')

    ROW_COUNTER = ROW_COUNTER + 1

    return row[0]['summary_text']

def get_syllabus(row,char_size):

    text = row['text']

    end_text = 'delivered the opinion of the Court'
    start_text = 'Syllabus'

    start_pos = text.find(start_text)
    end_pos = text.find(end_text)

    if (start_pos > 0 and end_pos > 0) and (end_pos > start_pos):
        text = text[start_pos+len(start_text):end_pos]

        if len(text) < int(char_size/2):
            text = text[start_pos+len(start_text):start_pos+char_size]

    elif start_pos < 0 and end_pos < 0:
        text = text[0:char_size] # o tokenizer trunca, nao tem problema aq
    elif start_pos > 0:
        text = text[start_pos+len(start_text):start_pos+char_size] # o tokenizer trunca, nao tem problema aq
    elif end_pos > 0:
        text = text[end_pos+len(end_text):end_pos+char_size] # o tokenizer trunca, nao tem problema aq
    else:
        text = text[0:char_size]

    if len(text) < int(char_size/2):
        text = row['text'][0:char_size]

    return text.strip()

#dfLawMark['text'] = dfLawMark.apply(lambda row : get_syllabus(row), axis=1)
dfLawMark['Syllabus'] = dfLawMark.apply(lambda row : summary_facebook(row), axis=1)
ROW_COUNTER = 0
dfLawMark['Syllabus_Legal'] = dfLawMark.apply(lambda row : summary_legal_text(row), axis=1)
ROW_COUNTER = 0

% Facebook completo : 0.0 
% Facebook completo : 0.25 
% Facebook completo : 0.51 
% Facebook completo : 0.76 
% Facebook completo : 1.02 
% Facebook completo : 1.27 
% Facebook completo : 1.53 
% Facebook completo : 1.78 
% Facebook completo : 2.04 
% Facebook completo : 2.29 
% Facebook completo : 2.54 
% Facebook completo : 2.8 
% Facebook completo : 3.05 
% Facebook completo : 3.31 
% Facebook completo : 3.56 
% Facebook completo : 3.82 
% Facebook completo : 4.07 
% Facebook completo : 4.33 
% Facebook completo : 4.58 
% Facebook completo : 4.83 
% Facebook completo : 5.09 
% Facebook completo : 5.34 
% Facebook completo : 5.6 
% Facebook completo : 5.85 
% Facebook completo : 6.11 
% Facebook completo : 6.36 
% Facebook completo : 6.62 
% Facebook completo : 6.87 
% Facebook completo : 7.12 
% Facebook completo : 7.38 
% Facebook completo : 7.63 
% Facebook completo : 7.89 
% Facebook completo : 8.14 
% Facebook completo : 8.4 
% Facebook completo : 8.65 
% Facebook completo : 8.

Your max_length is set to 512, but you input_length is only 483. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=241)


% Facebook completo : 34.1 
% Facebook completo : 34.35 
% Facebook completo : 34.61 
% Facebook completo : 34.86 
% Facebook completo : 35.11 
% Facebook completo : 35.37 
% Facebook completo : 35.62 
% Facebook completo : 35.88 
% Facebook completo : 36.13 
% Facebook completo : 36.39 
% Facebook completo : 36.64 
% Facebook completo : 36.9 
% Facebook completo : 37.15 
% Facebook completo : 37.4 
% Facebook completo : 37.66 
% Facebook completo : 37.91 
% Facebook completo : 38.17 
% Facebook completo : 38.42 
% Facebook completo : 38.68 
% Facebook completo : 38.93 
% Facebook completo : 39.19 
% Facebook completo : 39.44 
% Facebook completo : 39.69 
% Facebook completo : 39.95 
% Facebook completo : 40.2 
% Facebook completo : 40.46 
% Facebook completo : 40.71 
% Facebook completo : 40.97 
% Facebook completo : 41.22 
% Facebook completo : 41.48 
% Facebook completo : 41.73 
% Facebook completo : 41.98 
% Facebook completo : 42.24 
% Facebook completo : 42.49 
% Facebook complet

Your max_length is set to 512, but you input_length is only 475. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=237)


% Facebook completo : 61.83 
% Facebook completo : 62.09 
% Facebook completo : 62.34 
% Facebook completo : 62.6 
% Facebook completo : 62.85 
% Facebook completo : 63.1 
% Facebook completo : 63.36 
% Facebook completo : 63.61 
% Facebook completo : 63.87 
% Facebook completo : 64.12 
% Facebook completo : 64.38 
% Facebook completo : 64.63 
% Facebook completo : 64.89 
% Facebook completo : 65.14 
% Facebook completo : 65.39 
% Facebook completo : 65.65 
% Facebook completo : 65.9 
% Facebook completo : 66.16 
% Facebook completo : 66.41 
% Facebook completo : 66.67 
% Facebook completo : 66.92 
% Facebook completo : 67.18 
% Facebook completo : 67.43 
% Facebook completo : 67.68 
% Facebook completo : 67.94 
% Facebook completo : 68.19 
% Facebook completo : 68.45 
% Facebook completo : 68.7 
% Facebook completo : 68.96 
% Facebook completo : 69.21 
% Facebook completo : 69.47 
% Facebook completo : 69.72 
% Facebook completo : 69.97 
% Facebook completo : 70.23 
% Facebook complet

...

% Facebook completo : 74.81 
% Facebook completo : 75.06 
% Facebook completo : 75.32 
% Facebook completo : 75.57 
% Facebook completo : 75.83 
% Facebook completo : 76.08 
% Facebook completo : 76.34 
% Facebook completo : 76.59 
% Facebook completo : 76.84 
% Facebook completo : 77.1 
% Facebook completo : 77.35 
% Facebook completo : 77.61 
% Facebook completo : 77.86 
% Facebook completo : 78.12 
% Facebook completo : 78.37 
% Facebook completo : 78.63 
% Facebook completo : 78.88 
% Facebook completo : 79.13 
% Facebook completo : 79.39 
% Facebook completo : 79.64 
% Facebook completo : 79.9 
% Facebook completo : 80.15 
% Facebook completo : 80.41 
% Facebook completo : 80.66 
% Facebook completo : 80.92 
% Facebook completo : 81.17 
% Facebook completo : 81.42 
% Facebook completo : 81.68 
% Facebook completo : 81.93 
% Facebook completo : 82.19 
% Facebook completo : 82.44 
% Facebook completo : 82.7 
% Facebook completo : 82.95 
% Facebook completo : 83.21 
% Facebook comple

Your max_length is set to 512, but you input_length is only 511. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=255)


% Facebook completo : 85.75 
% Facebook completo : 86.01 
% Facebook completo : 86.26 
% Facebook completo : 86.51 
% Facebook completo : 86.77 
% Facebook completo : 87.02 
% Facebook completo : 87.28 
% Facebook completo : 87.53 
% Facebook completo : 87.79 
% Facebook completo : 88.04 
% Facebook completo : 88.3 
% Facebook completo : 88.55 
% Facebook completo : 88.8 
% Facebook completo : 89.06 
% Facebook completo : 89.31 
% Facebook completo : 89.57 
% Facebook completo : 89.82 
% Facebook completo : 90.08 
% Facebook completo : 90.33 
% Facebook completo : 90.59 
% Facebook completo : 90.84 
% Facebook completo : 91.09 
% Facebook completo : 91.35 
% Facebook completo : 91.6 


Your max_length is set to 512, but you input_length is only 511. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=255)


% Facebook completo : 91.86 
% Facebook completo : 92.11 
% Facebook completo : 92.37 
% Facebook completo : 92.62 
% Facebook completo : 92.88 
% Facebook completo : 93.13 
% Facebook completo : 93.38 
% Facebook completo : 93.64 
% Facebook completo : 93.89 
% Facebook completo : 94.15 
% Facebook completo : 94.4 
% Facebook completo : 94.66 
% Facebook completo : 94.91 
% Facebook completo : 95.17 
% Facebook completo : 95.42 
% Facebook completo : 95.67 
% Facebook completo : 95.93 
% Facebook completo : 96.18 
% Facebook completo : 96.44 
% Facebook completo : 96.69 
% Facebook completo : 96.95 
% Facebook completo : 97.2 
% Facebook completo : 97.46 
% Facebook completo : 97.71 
% Facebook completo : 97.96 
% Facebook completo : 98.22 
% Facebook completo : 98.47 
% Facebook completo : 98.73 
% Facebook completo : 98.98 
% Facebook completo : 99.24 
% Facebook completo : 99.49 
% Facebook completo : 99.75 
% Indiano completo : 0.0 
% Indiano completo : 0.25 
% Indiano completo : 

In [5]:
dfLawMark.head(5)

,text,subject,Syllabus,Syllabus_Legal
1166,"461 U.S 103 S.Ct 75 L.Ed.2d 708 Harry CONNICK,...",Freedom of Speech,Assistant District Attorney distributed a ques...,The U.S. District Court for the Southern Distr...
1099,"354 U.S 77 S.Ct 1 L.Ed.2d 1498 Samuel ROTH, Pe...",Freedom of Speech,"In Alberts, the primary constitutional questio...",The U.S. District Court for the Southern Distr...
836,403 U.S 91 S.Ct 29 L.Ed.2d 745 Alton J. LEMON ...,First Amendment,Rhode Island's 1969 Salary Supplement Act pro...,The U.S. District Court for the District of Rh...
789,"297 U.S 56 S.Ct 80 L.Ed GROSJEAN, Sup'r of Pub...",First Amendment,Nine publishers of newspapers in the state of ...,The U.S. District Court for the Eastern Distri...
793,310 U.S 60 S.Ct 84 L.Ed CANTWELL et al.v.STATE...,First Amendment,"Newton Cantwell and his two sons, Jesse and Ru...",The U.S. District Court for the Middle Distric...


In [6]:
encodedClass = pd.factorize(dfLawMark['subject'])
dfLawMark['encoded'] = encodedClass[0]+1 ## pytorch precisa começar do 1

dfEnCode = dfLawMark[['subject','encoded']]
dfEnCode.drop_duplicates(inplace=True)

dfEnCode.to_csv('/home/jaco/Projetos/landMarkClassification/data/enCode.csv',index=False)
dfLawMark.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus.csv',index=False)

/tmp/ipykernel_4948/2678490174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfEnCode.drop_duplicates(inplace=True)


In [7]:
from sklearn.model_selection import train_test_split

X = dfLawMark[['Syllabus','Syllabus_Legal']]
y = dfLawMark['encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=666, stratify=y)

dfLawMark_Train = pd.DataFrame()
dfLawMark_Train['Syllabus'] = X_train.iloc[: , 0:1]
dfLawMark_Train['Syllabus_Legal'] = X_train.iloc[: , 1:2]
dfLawMark_Train['encoded'] = y_train.to_frame()

dfLawMark_Test = pd.DataFrame()
dfLawMark_Test['Syllabus'] = X_test.iloc[: , 0:1]
dfLawMark_Test['Syllabus_Legal'] = X_test.iloc[: , 1:2]
dfLawMark_Test['encoded'] = y_test.to_frame()

dfLawMark_Train.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Train.csv',index=False)
print(f'tamanho do train dataset: {len(dfLawMark_Train)}')
dfLawMark_Test.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Test.csv',index=False)
print(f'tamanho do test dataset: {len(dfLawMark_Test)}')

tamanho do train dataset: 353
tamanho do test dataset: 40


In [8]:
#ag_test = pd.read_csv('/home/jaco/Projetos/landMarkClassification/data/agnews/test.csv')
#ag_train = pd.read_csv('/home/jaco/Projetos/landMarkClassification/data/agnews/train.csv')

#ag_test = ag_test.sample(n=300)
#ag_train = ag_test.sample(n=100)

#ag_test = ag_test.to_csv('/home/jaco/Projetos/landMarkClassification/data/agnews/test_red.csv',index=False)
#ag_train = ag_train.to_csv('/home/jaco/Projetos/landMarkClassification/data/agnews/train_red.csv',index=False)

In [9]:
text = '''401 U.S. 424 91 S.Ct. 849 28 L.Ed.2d 158 Willie S. GRIGGS et al., Petitioners,v.DUKE POWER COMPANY. No. 124. Argued Dec. 14, 1970. Decided March 8, 1971.  Jack Greenberg, New York City, for petitioners. George W. Ferguson, Jr., for respondent. Lawrence M. Cohen for the Chamber of Commerce of the United States, as amicus curiae. Mr. Chief Justice BURGER delivered the opinion of the Court.   1 We granted the writ in this case to resolve the question whether an employer is prohibited by the Civil Rights Act of 1964, Title VII, from requiring a high school education or passing of a standardized general intelligence test as a condition of employment in or transfer to jobs when (a) neither standard is shown to be significantly related to successful job performance, (b) both requirements operate to disqualify Negroes at a substantially higher rate than white applicants, and (c) the jobs in question formerly had been filled only by white employees as part of a longstanding practice of giving preference to whites.1   2 Congress provided, in Title VII of the Civil Rights Act of 1964, for class actions for enforcement of provisions of the Act and this proceeding was brought by a group of incumbent Negro employees against Duke Power Company. All the petitioners are employed at the Company's Dan River Steam Station, a power generating facility located at Draper, North Carolina. At the time this action was instituted, the Company had 95 employees at the Dan River Station, 14 of whom were Negroes; 13 of these are petitioners here.   3 The District Court found that prior to July 2, 1965, the effective date of the Civil Rights Act of 1964, the Company openly discriminated on the basis of race in the hiring and assigning of employees at its Dan River plant. The plant was organized into five operating departments: (1) Labor, (2) Coal Handling, (3) Operations, (4) Maintenance, and (5) Laboratory and Test. Negroes were employed only in the Labor Department where the highest paying jobs paid less than the lowest paying jobs in the other four 'operating' departments in which only whites were employed.2 Promotions were normally made within each department on the basis of job seniority. Transferees into a department usually began in the lowest position.   4 In 1955 the Company instituted a policy of requiring a high school education for initial assignment to any department except Labor, and for transfer from the Coal Handling to any 'inside' department (Operations, Maintenance, or Laboratory). When the Company abandoned its policy of restricting Negroes to the Labor Department in 1965, completion of high school also was made a prerequisite to transfer from Labor to any other department. From the time the high school requirement was instituted to the time of trial, however, white employees hired before the time of the high school education requirement continued to perform satisfactorily and achieve promotions in the 'operating' departments. Findings on this score are not challenged.   5 The Company added a further requirement for new employees on July 2, 1965, the date on which Title VII became effective. To qualify for placement in any but the Labor Department it become necessary to register satisfactory scores on two professionally prepared aptitude tests, as well as to have a high school education. Completion of high school alone continued to render employees eligible for transfer to the four desirable departments from which Negroes had been excluded if the incumbent had been employed prior to the time of the new requirement. In September 1965 the Company began to permit incumbent employees who lacked a high school education to qualify for transfer from Labor or Coal Handling to an 'inside' job by passing two tests—the Wonderlic Personnel Test, which purports to measure general intelligence, and the Bennett Mechanical Comprehension Test. Neither was directed or intended to measure the ability to learn to perform a particular job or category of jobs. The requisite scores used for both initial hiring and transfer approximated the national median for high school graduates.3   6 The District Court had found that while the Company previously followed a policy of overt racial discrimination in a period prior to the Act, such conduct had ceased. The District Court also concluded that Title VII was intended to be prospective only and, consequently, the impact of prior inequities was beyond the reach of corrective action authorized by the Act.   7 The Court of Appeals was confronted with a question of first impression, as are we, concerning the meaning of Title VII. After careful analysis a majority of that court concluded that a subjective test of the employer's intent should govern, particularly in a close case, and that in this case there was no showing of a discriminatory purpose in the adoption of the diploma and test requirements. On this basis, the Court of Appeals concluded there was no violation of the Act.   8 The Court of Appeals reversed the District Court in part, rejecting the holding that residual discrimination arising from prior employment practices was insulated from remedial action.4 The Court of Appeals noted, however, that the District Court was correct in its conclusion that there was no showing of a racial purpose or invidious intent in the adoption of the high school diploma requirement or general intelligence test and that these standards had been applied fairly to whites and Negroes alike. It held that, in the absence of a discriminatory purpose, use of such requirements was permitted by the Act. In so doing, the Court of Appeals rejected the claim that because these two requirements operated to render ineligible a markedly disproportionate number of Negroes, they were unlawful under Title VII unless shown to be job related.5 We granted the writ on these claims. 399 U.S. 926, 90 S.Ct. 2238, 26 L.Ed.2d 791.   9 The objective of Congress in the enactment of Title VII is plain from the language of the statute. It was to achieve equality of employment opportunities and remove barriers that have operated in the past to favor an identifiable group of white employees over other employees. Under the Act, practices, procedures, or tests neutral on their face, and even neutral in terms of intent, cannot be maintained if they operate to 'freeze' the status quo of prior discriminatory employment practices.   10 The Court of Appeals' opinion, and the partial dissent, agreed that, on the record in the present case, 'whites register far better on the Company's alternative requirements' than Negroes.6 420 F.2d 1225, 1239 n. 6. This consequence would appear to be directly traceable to race. Basic intelligence must have the means of articulation to manifest itself fairly in a testing process. Because they are Negroes, petitioners have long received inferior education in segregated schools and this Court expressly recognized these differences in Gaston County v. United States, 395 U.S. 285, 89 S.Ct. 1720, 23 L.Ed.2d 309 (1969). There, because of the inferior education received by Negroes in North Carolina, this Court barred the institution of a literacy test for voter registration on the ground that the test would abridge the right to vote indirectly on account of race. Congress did not intend by Title VII, however, to guarantee a job to every person regardless of qualifications. In short, the Act does not command that any person be hired simply because he was formerly the subject of discrimination, or because he is a member of a minority group. Discriminatory preference for any group, minority or majority, is precisely and only what Congress has proscribed. What is required by Congress is the removal of artificial, arbitrary, and unnecessary barriers to employment when the barriers operate invidiously to discriminate on the basis of racial or other impermissible classification.   11 Congress has now provided that tests or criteria for employment or promotion may not provide equality of opportunity merely in the sense of the fabled offer of milk to the stork and the fox. On the contrary, Congress has now required that the posture and condition of the job-seeker be taken into account. It has—to resort again to the fable—provided that the vessel in which the milk is proffered be one all seekers can use. The Act proscribes not only overt discrimination but also practices that are fair in form, but discriminatory in operation. The touchstone is business necessity. If an employment practice which operates to exclude Negroes cannot be shown to be related to job performance, the practice is prohibited.   12 On the record before us, neither the high school completion requirement nor the general intelligence test is shown to bear a demonstrable relationship to successful performance of the jobs for which it was used. Both were adopted, as the Court of Appeals noted, without meaningful study of their relationship to job-performance ability. Rather, a vice president of the Company testified, the requirements were instituted on the Company's judgment that they generally would improve the overall quality of the work force.   13 The evidence, however, shows that employees who have not completed high school or taken the tests have continued to perform satisfactorily and make progress in departments for which the high school and test criteria are now used.7 The promotion record of present employees who would not be able to meet the new criteria thus suggests the possibility that the requirements may not be needed even for the limited purpose of preserving the avowed policy of advancement within the Company. In the context of this case, it is unnecessary to reach the question whether testing requirements that take into account capability for the next succeeding position or related future promotion might be utilized upon a showing that such longrange requirements fulfill a genuine business need. In the present case the Company has made no such showing.   14 The Court of Appeals held that the Company had adopted the diploma and test requirements without any 'intention to discriminate against Negro employees.' 420 F.2d, at 1232. We do not suggest that either the District Court or the Court of Appeals erred in examining the employer's intent; but good intent or absence of discriminatory intent does not redeem employment procedures or testing mechanisms that operate as 'built-in headwinds' for minority groups and are unrelated to measuring job capability.   15 The Company's lack of discriminatory intent is suggested by special efforts to help the undereducated employees through Company financing of two-thirds the cost of tuition for high school training. But Congress directed the thrust of the Act to the consequences of employment practices, not simply the motivation. More than that, Congress has placed on the employer the burden of showing that any given requirement must have a manifest relationship to the employment in question.   16 The facts of this case demonstrate the inadequacy of broad and general testing devices as well as the infirmity of using diplomas or degrees as fixed measures of capability. History is filled with examples of men and women who rendered highly effective performance without the conventional badges of accomplishment in terms of certificates, diplomas, or degrees. Diplomas and tests are useful servants, but Congress has mandated the commonsense proposition that they are not to become masters of reality.   17 The Company contends that its general intelligence tests are specifically permitted by § 703(h) of the Act.8 That section authorizes the use of 'any professionally developed ability test' that is not 'designed, intended or used to discriminate because of race * * *.' (Emphasis added.)   18 The Equal Employment Opportunity Commission, having enforcement responsibility, has issued guidelines interpreting § 703(h) to permit only the use of job-related tests.9 The administrative interpretation of the Act by the enforcing agency is entitled to great deference. See, e.g., United States v. City of Chicago, 400 U.S. 8, 91 S.Ct. 18, 27 L.Ed.2d 9 (1970); Udall v. Tallman, 380 U.S. 1, 85 S.Ct. 792, 13 L.Ed.2d 616 (1965); Power Reactor Development Co. v. Electricians, 367 U.S. 396, 81 S.Ct. 1529, 6 L.Ed.2d 924 (1961). Since the Act and its legislative history support the Commission's construction, this affords good reason to treat the guidelines as expressing the will of Congress.   19 Section 703(h) was not contained in the House version of the Civil Rights Act but was added in the Senate during extended debate. For a period, debate revolved around claims that the bill as proposed would prohibit all testing and force employers to hire unqualified persons simply because they were part of a group formerly subject to job discrimination.10 Proponents of Title VII sought throughout the debate to assure the critics that the Act would have no effect on job-related tests. Senators Case of New Jersey and Clark of Pennsylvania, comanagers of the bill on the Senate floor, issued a memorandum explaining that the proposed Title VII 'expressly protects the employer's right to insist that any prospective applicant, Negro or white, must meet the applicable job qualifications. Indeed, the very purpose of title VII is to promote hiring on the basis of job qualifications, rather than on the basis of race or color.' 110 Cong.Rec. 7247.11 (Emphasis added.) Despite these assurances, Senator Tower of Texas introduced an amendment authorizing 'professionally developed ability tests.' Proponents of Title VII opposed the amendment because, as written, it would permit an employer to give any test, 'whether it was a good test or not, so long as it was professionally designed. Discrimination could actually exist under the guise of compliance with the statute.' 110 Cong.Rec. 13504 (remarks of Sen. Case).   20 The amendment was defeated and two days later Senator Tower offered a substitute amendment which was adopted verbatim and is now the testing provision of § 703(h). Speaking for the supporters of Title VII, Senator Humphrey, who had vigorously opposed the first amendment, endorsed the substitute amendment, stating: 'Senators on both sides of the aisle who were deeply interested in title VII have examined the text of this amendment and have found it to be in accord with the intent and purpose of that title.' 110 Cong.Rec. 13724. The amendment was then adopted.12 From the sum of the legislative history relevant in this case, the conclusion is inescapable that the EEOC's construction of § 703(h) to require that employment tests be job related comports with congressional intent.   21 Nothing in the Act precludes the use of testing or measuring procedures; obviously they are useful. What Congress has forbidden is giving these devices and mechanisms controlling force unless they are demonstrably a reasonable measure of job performance. Congress has not commanded that the less qualified be preferred over the better qualified simply because of minority origins. Far from disparaging job qualifications as such, Congress has made such qualifications the controlling factor, so that race, religion, nationality, and sex become irrelevant. What Congress has commanded is that any tests used must measure the person for the job and not the person in the abstract.   22 The judgment of the Court of Appeals is, as to that portion of the judgment appealed from, reversed.   23 Mr. Justice BRENNAN took no part in the consideration or decision of this case.    1  The Act provides: 'Sec. 703. (a) It shall be an unlawful employment practice for an employer— '(2) to limit, segregate, or classify his employees in any way which would deprive or tend to deprive any individual of employment opportunities or otherwise adversely affect his status as an employee, because of such individual's race, color, religion, sex, or national origin. '(h) Notwithstanding any other provision of this title, it shall not be an unlawful employment practice for an employer * * * to give and to act upon the results of any professionally developed ability test provided that such test, its administration or action upon the results is not designed, intended or used to discriminate because of race, color, religion, sex or national origin. * * *' 78 Stat. 255, 42 U.S.C. § 2000e—2.   2  A Negro was first assigned to a job in an operating department in August 1966, five months after charges had been filed with the Equal Employment Opportunity Commission. The employee, a high school graduate who had begun in the Labor Department in 1953, was promoted to a job in the Coal Handling Department.   3  The test standards are thus more stringent than the high school requirement, since they would screen out approximately half of all high school graduates.   4  The Court of Appeals ruled that Negroes employed in the Labor Department at a time when there was no high school or test requirement for entrance into the higher paying departments could not now be made subject to those requirements, since whites hired contemporaneously into those departments were never subject to them. The Court of Appeals also required that the seniority rights of those Negroes be measured on a plantwide, rather than a departmental, basis. However, the Court of Appeals denied relief to the Negro employees without a high school education or its equivalent who were hired into the Labor Department after institution of the educational requirement.   5  One member of that court disagreed with this aspect of the decision, maintaining, as do the petitioners in this Court, that Title VII prohibits the use of employment criteria that operate in a racially exclusionary fashion and do not measure skills or abilities necessary to performance of the jobs for which those criteria are used   6  In North Carolina, 1960 census statistics show that, while 34% of white males had completed high school, only 12% of Negro males had done so. U.S. Bureau of the Census, U.S. Census of Population: 1960, Vol. 1, Characteristics of the Population, pt. 35, Table 47. Similarly, with respect to standardized tests, the EEOC in one case found that use of a battery of tests, including the Wonderlic and Bennett tests used by the Company in the instant case, resulted in 58% of whites passing the tests, as compared with only 6% of the blacks. Decision of EEOC, CCH Empl.Prac. Guide, 17,304.53 (Dec. 2, 1966). See also Decision of EEOC 70—552, CCH Empl.Prac. Guide, 6139 (Feb. 19, 1970).   7  For example, between July 2, 1965, and November 14, 1966, the percentage of white employees who were promoted but who were not high school graduates was nearly identical to the percentage of nongraduates in the entire white work force.   8  Section 703(h) applies only to tests. It has no applicability to the high school diploma requirement.   9  EEOC Guidelines on Employment Testing Procedures, issued August 24, 1966, provide: 'The Commission accordingly interprets 'professionally developed ability test' to mean a test which fairly measures the knowledge or skills required by the particular job or class of jobs which the applicant seeks, or which fairly affords the employer a chance to measure the applicant's ability to perform a particular job or class of jobs. The fact that a test was prepared by an individual or organization claiming expertise in test preparation does not, without more, justify its use within the meaning of Title VII.' The EEOC position has been elaborated in the new Guidelines on Employee Selection Procedures, 29 CFR § 1607, 35 Fed.Reg. 12333 (Aug. 1, 1970). These guidelines demand that employers using tests have available 'data demonstrating that the test is predictive of or significantly correlated with important elements of work behavior which comprise or are relevant to the job or jobs for which candidates are being evaluated.' Id., at § 1607.4(c).   10  The congressional discussion was prompted by the decision of a hearing examiner for the Illinois Fair Employment Commission in Myart v. Motorola Co. (The decision is reprinted at 110 Cong.Rec. 5662.) That case suggested that standardized tests on which whites performed better than Negroes could never be used. The decision was taken to mean that such tests could never be justified even if the needs of the business required them. A number of Senators feared that Title VII might produce a similar result. See remarks of Senators Ervin, 110 Cong.Rec. 5614—5616; Smathers, id., at 5999—6000; Holland, id., at 7012—7013; Hill, id., at 8447; Tower, id., at 9024; Talmadge, id., at 9025—9026; Fulbright, id., at 9599—9600; and Ellender, id., at 9600.   11  The Court of Appeals majority, in finding no requirement in Title VII that employment tests be job related, relied in part on a quotation from an earlier Clark-Case interpretative memorandum addressed to the question of the constitutionality of Title VII. The Senators said in that memorandum: 'There is no requirement in title VII that employers abandon bona fide qualification tests where, because of differences in background and education, members of some groups are able to perform better on these tests than members of other groups. An employer may set his qualifications as high as he likes, he may test to determine which applicants have these qualifications, and he may hire, assign, and promote on the basis of test performance.' 110 Cong.Rec. 7213. However, nothing there stated conflicts with the later memorandum dealing specifically with the debate over employer testing, 110 Cong.Rec. 7247 (quoted from in the text above), in which Senators Clark and Case explained that tests which measure 'applicable job qualifications' are permissible under Title VII. In the earlier memorandum Clark and Case assured the Senate that employers were not to be prohibited from using tests that determine qualifications. Certainly a reasonable interpretation of what the Senators meant, in light of the subsequent memorandum directed specifically at employer testing, was that nothing in the Act prevents employers from requiring that applicants be fit for the job.   12  Senator Tower's original amendment provided in part that a test would be permissible 'if * * * in the case of any individual who is seeking employment with such employer, such test is designed to determine or predict whether such individual is suitable or trainable with respect to his employment in the particular business or enterprise involved * * *.' 110 Cong.Rec. 13492. This language indicates that Senator Tower's aim was simply to make certain that job-related tests would be permitted. The opposition to the amendment was based on its loose wording which the proponents of Title VII feared would be susceptible of misinterpretation. The final amendment, which was acceptable to all sides, could hardly have required less of a job relation than the first'''

In [10]:
#row = text
#row = tokenizerFaceBook(row,padding='max_length', max_length = int((1024)*0.95), truncation=True, return_tensors="pt")
#row = tokenizerFaceBook.decode(row.input_ids[0], skip_special_tokens=True)
#print(row)
#row = summarizerFacebook(row, max_length=512, min_length=256, do_sample=False)

#print(f'% completada : {round(ROW_COUNTER/DATSET_SIZE*100,2)} ')

#ROW_COUNTER = ROW_COUNTER + 1

#print(row[0]['summary_text'])